In [20]:
import pandas as pd
import re
from sqlalchemy import create_engine

In [44]:
class ETL():
    def __init__(self) -> None:
        pass

    def connection(
            self,
            username: str,
            password: str,
            hostname: str,
            port: int,
            database: str,
            database_type: str
            ):
        engine = create_engine(f"{database_type}://{username}:{password}@{hostname}:{port}/{database}")
        return engine

    def read_data(
            self,
            conn, 
            tables_name: list
            ):
        dfs = []
        for table in tables_name:
            query = f"SELECT * FROM {table}"
            df = pd.read_sql(query, conn)
            dfs.append(df)
        return dfs
    
    def merge_data(
        self, 
        dfs, 
        key
        ):
        if len(dfs) > 1:
            df = dfs[0].merge(dfs[1], how='left', on=key)
            df['title'] = df['title'].astype(str)
            return df
        return df[0]
    
    def get_year(self, title):
        try:
            return re.search(r'\((\d{4})|-\)', str(title)).group(1)
        except Exception as e:
            print(title,': ', e)
            return None

    def cleaning_data(
        self, 
        df
        ):
        df.drop_duplicates(inplace=True)
        df.dropna(inplace=True)
        df['year'] = df['title'].apply(self.get_year)
        df['title'] = df['title'].apply(lambda x: re.sub(r'\s*\(\d{4}\)$', '', x))
        df['genres'] = df['genres'].apply(lambda x: f"{[f'{i}' for i in x.split('|')]}")
        return df

    def transform_data(
        self, 
        df, 
        ):
        agg_funcs = { 
            'title': pd.Series.mode,
            'genres': pd.Series.mode,
            'year': pd.Series.mode,
            'rating': pd.Series.mean,
        }
        df = df.groupby('movieId').agg(agg_funcs).reset_index()
        
        df.rename(columns={
            'movieId': 'movie_id',
            'rating': 'rating_avg',
            'genres': 'genre'
        }, inplace=True)
        return df[['movie_id', 'title', 'year', 'genre', 'rating_avg']]

    def load_data(
        self,
        df,
        conn,
        table_name,
        schema=None
        ):
        df.to_sql(table_name, conn, if_exists='append', schema=schema, index=False)

    

In [45]:
etl = ETL()
conn = etl.connection(
    username='root', 
    password='', 
    hostname='localhost', 
    port=3306, 
    database='movielens_movie', 
    database_type='mysql+pymysql'
    )

In [46]:
dfs = etl.read_data(conn=conn, tables_name=['movie', 'rating'])
#dfs

In [47]:
df = etl.merge_data(dfs, key='movieId')
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [48]:
df = etl.cleaning_data(df=df)
df.head()

Babylon 5 :  'NoneType' object has no attribute 'group'


,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",3.0,4.0,1999-12-11 13:36:47,1995
1,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",6.0,5.0,1997-03-13 17:50:52,1995
2,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",8.0,4.0,1996-06-05 13:37:51,1995
3,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",10.0,4.0,1999-11-25 02:44:47,1995
4,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",11.0,4.5,2009-01-02 01:13:41,1995


In [49]:
df = etl.transform_data(df=df)
df.head()

,movie_id,title,year,genre,rating_avg
0,1,Toy Story,1995,"['Adventure', 'Animation', 'Children', 'Comedy...",3.956032
1,2,Jumanji,1995,"['Adventure', 'Children', 'Fantasy']",3.264535
2,3,Grumpier Old Men,1995,"['Comedy', 'Romance']",3.190713
3,4,Waiting to Exhale,1995,"['Comedy', 'Drama', 'Romance']",3.063025
4,5,Father of the Bride Part II,1995,['Comedy'],3.137755


In [50]:
etl.load_data(conn=conn, df=df, table_name='agg')

In [42]:
a = 'a|b|c|d|e|f|g|h'
c = [f'{x}' for x in a.split('|')]

In [43]:
f"{c}"

"['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']"

## Data Info

In [21]:
df.describe()

,movieId,userId,rating
count,943406.000000,929946.000000,929946.000000
mean,9851.582636,3125.981875,3.526725
std,21620.581566,1778.465857,1.051058
min,1.000000,1.000000,0.500000
25%,912.000000,1602.000000,3.000000
50%,2203.000000,3177.000000,4.000000
75%,4885.000000,4620.000000,4.000000
max,131262.000000,6217.000000,5.000000


In [23]:
df.duplicated().sum()

0

In [28]:
# Show rows with NaN values
rows_with_nan = df[df.isna().any(axis=1)]
rows_with_nan

,movieId,title,genres,userId,rating,timestamp
36714,98,Shopping (1994),Action|Thriller,NaN,NaN,NaN
43088,115,Happiness Is in the Field (Bonheur est dans le...,Comedy,NaN,NaN,NaN
45546,143,Gospa (1995),Drama,NaN,NaN,NaN
131357,395,Desert Winds (1995),Drama|Fantasy|Romance,NaN,NaN,NaN
131373,401,Mirage (1995),Action|Thriller,NaN,NaN,NaN
...,...,...,...,...,...,...
943401,131254,Kein Bund für's Leben (2007),Comedy,NaN,NaN,NaN
943402,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,NaN,NaN,NaN
943403,131258,The Pirates (2014),Adventure,NaN,NaN,NaN
943404,131260,Rentun Ruusu (2001),(no genres listed),NaN,NaN,NaN


In [29]:
df['title'].value_counts()

Pulp Fiction (1994)                   3075
Forrest Gump (1994)                   3067
Silence of the Lambs, The (1991)      2846
Shawshank Redemption, The (1994)      2838
Jurassic Park (1993)                  2746
                                      ... 
Telstar: The Joe Meek Story (2008)       1
In A Day (2006)                          1
Name for Evil, A (1973)                  1
8: The Mormon Proposition (2010)         1
Innocence (2014)                         1
Name: title, Length: 27259, dtype: int64